# Modificar la simulacion de eventos discretos 

### Agregar parametros

1. Agregar y modificar el tiempo que se demora de transportar de la cola de espera a la maquina de lavado dentro de (1-5)
2. Identificar el numero de la maquina utilizando dentro de cada proceso 
3. Agregar y modificar el tiempo de salir de la maquina a la puerta principal del negocio entre (2-5)

In [1]:
import simpy
import random

In [2]:
def manejar_ahora(env, carro):
    yield env.timeout(random.randint(1,20))
    carro.accion.interrupt()

In [3]:
duracion_carga = 5
duracion_bateria = 2

class Carro():
    def __init__(self, env):
        self.env = env
        self.accion = env.process(self.run())
        
    def run(self):
        while True:
            print('Se parquea y empieza la recarga %d' % self.env.now)
            try:
                yield self.env.process(self.recarga())
            except simpy.Interrupt:            
                print('Se interrumpe carga %d' % self.env.now)
                
            print('Se termina la recarga %d y empieza el manejo' % self.env.now)
            yield self.env.timeout(duracion_bateria)
            print('Uso del carro manejo %d' % self.env.now)

    def recarga(self):
        yield self.env.timeout(duracion_carga)

In [4]:
env = simpy.Environment()
carro1 = Carro(env)
env.process(manejar_ahora(env, carro1))
env.run(until=25)

Se parquea y empieza la recarga 0
Se termina la recarga 5 y empieza el manejo


Interrupt: Interrupt(None)

In [ ]:
TIEMPO_FABRICACION = 10
TIEMPO_FALLA = 300
TIEMPO_REPARACION = 38
TIEMPO_OTROS_TRABAJOS = 30
NUM_MAQUINAS = 10
SEMANAS = 4
TIEMPO_SIMULACION = SEMANAS * 7 * 24 * 60

In [ ]:
def tiempo_fabricacion():
    return random.normalvariate(TIEMPO_FABRICACION, 2)

def tiempo_falla():
    return random.expovariate(1/TIEMPO_FALLA)

In [ ]:
class Maquina():
    def __init__(self, env, nombre, tecnico):
        self.env = env
        self.nombre = nombre
        self.numero_piezas = 0
        self.falla = False
        self.numero_fallas = 0
        self.proceso = env.process(self.trabajo(tecnico))
        env.process(self.falla_maquina())
    
    def trabajo(self, tecnico):
        while True:
            demora_pieza = tiempo_fabricacion()
            while demora_pieza:
                try:
                    inicio = self.env.now
                    yield self.env.timeout(demora_pieza)
                    demora_pieza = 0
                except simpy.Interrupt:
                    self.numero_fallas += 1
                    self.falla = True
                    demora_pieza -= self.env.now - inicio
                    with tecnico.request(priority=1) as requerimiento:
                        yield requerimiento
                        yield self.env.timeout(TIEMPO_REPARACION)
                    self.falla = False
                self.numero_piezas += 1
    def falla_maquina(self):
        while True:
            yield self.env.timeout(tiempo_falla())
            if not self.falla:
                self.proceso.interrupt()

In [ ]:
def trabajo_tecnico(env, tecnico):
    while True:
        tiempo = TIEMPO_OTROS_TRABAJOS
        while tiempo:
            with tecnico.request(priority=2) as requerimiento:
                yield requerimiento;
                try:
                    inicio = env.now
                    yield env.timeout(tiempo)
                    tiempo = 0
                except simpy.Interrupt:
                    tiempo -= env.now - inicio

In [ ]:
print("Fabriación Iniciada...")

env = simpy.Environment()
tecnico = simpy.PreemptiveResource(env, capacity=1)
maquinas = [Maquina(env, 'Máquina %d' % 1, tecnico) for i in range(NUM_MAQUINAS)]
env.process(trabajo_tecnico(env, tecnico))
env.run(until=TIEMPO_SIMULACION)
print("Simulación realizada despues de %d semanas" % (SEMANAS))

for maquina in maquinas:
    print('Nombre: %s ha realizado %d partes con numero de fallas %d' % (maquina.nombre, maquina.numero_piezas, maquina.numero_fallas))